# Usar tabular env

In [ ]:
# Bibliotecas
import pandas as pd
import geopandas as gpd
import folium
from folium import GeoJson
import json


# Definir cores corretas para cada classe
class_colors = {
    0: '#bf8040',  # cor_bare
    1: '#009900',  # cor_bush
    2: '#ffdf99',  # cor_crop
    3: '#33ff33',  # cor_grass
    4: '#69000d',  # cor_hduf
    5: '#e93529',  # cor_industrial
    6: '#c7171c',  # cor_lduf
    7: '#a10e15',  # cor_mduf
    8: '#003300',  # cor_tree
    9: '#0b9fd5'   # cor_water
}

# Labels para cada classe
class_labels = {
    0: 'Bare soil',
    1: 'Shrub/Scrub',
    2: 'Row Crops',
    3: 'Grass/Pasture',
    4: 'Developed High Density',
    5: 'Developed Industrial',
    6: 'Developed Low Density',
    7: 'Developed Medium Density',
    8: 'Perenial Trees',
    9: 'Perenial Water'
}

# Dados das cidades
cities_data = {
    'Apucarana': {
        'path': '/Volumes/ssd_externo/UEL DOUTORADO 2022/Artigo CLIP GEO/Apucarana/classificacaoApucarana.shp',
        'areas': {0: '4.31 (3.99%)', 1: '4.43 (4.1%)', 2: '37.13 (34.36%)', 3: '8.87 (8.21%)', 
                 4: '0.68 (0.63%)', 5: '7.18 (6.65%)', 6: '18.24 (16.87%)', 7: '8.46 (7.83%)', 
                 8: '18.17 (16.81%)', 9: '0.6 (0.55%)'},
        'total': '108.07'
    },
    'Arapongas': {
        'path': '/Volumes/ssd_externo/UEL DOUTORADO 2022/Artigo CLIP GEO/Arapongas/classificacaoArapongas.shp',
        'areas': {0: '4.57 (3.58%)', 1: '1.65 (1.29%)', 2: '55.88 (43.82%)', 3: '17.44 (13.67%)', 
                 4: '0.35 (0.27%)', 5: '10.39 (8.14%)', 6: '17.59 (13.79%)', 7: '7.6 (5.96%)', 
                 8: '11.72 (9.19%)', 9: '0.35 (0.27%)'},
        'total': '127.53'
    },
    'Cambe': {
        'path': '/Volumes/ssd_externo/UEL DOUTORADO 2022/Artigo CLIP GEO/Cambe/classificacaoCambe.shp',
        'areas': {0: '5.57 (7.87%)', 1: '2.25 (3.18%)', 2: '26.52 (37.49%)', 3: '2.58 (3.65%)', 
                 4: '0.16 (0.22%)', 5: '6.44 (9.11%)', 6: '12.27 (17.35%)', 7: '9.67 (13.67%)', 
                 8: '4.79 (6.78%)', 9: '0.48 (0.68%)'},
        'total': '70.72'
    },
    'Sarandi': {
        'path': '/Volumes/ssd_externo/UEL DOUTORADO 2022/Artigo CLIP GEO/Sarandi/classificacaoSarandi.shp',
        'areas': {0: '1.55 (5.55%)', 1: '0.26 (0.94%)', 2: '5.04 (18.02%)', 3: '1.67 (5.98%)', 
                 4: '0.01 (0.04%)', 5: '2.8 (9.99%)', 6: '13.99 (50.0%)', 7: '2.32 (8.28%)', 
                 8: '0.32 (1.15%)', 9: '0.01 (0.04%)'},
        'total': '27.99'
    },
    'Cambira': {
        'path': '/Volumes/ssd_externo/UEL DOUTORADO 2022/Artigo CLIP GEO/Cambira/classificacaoCambira.shp',
        'areas': {0: '0.13 (4.62%)', 1: '0.0 (0.0%)', 2: '0.38 (13.45%)', 3: '0.05 (1.68%)', 
                 4: '0.0 (0.0%)', 5: '0.3 (10.5%)', 6: '1.28 (44.95%)', 7: '0.63 (22.27%)', 
                 8: '0.07 (2.52%)', 9: '0.0 (0.0%)'},
        'total': '2.84'
    },
    'Ibipora': {
        'path': '/Volumes/ssd_externo/UEL DOUTORADO 2022/Artigo CLIP GEO/Ibipora/classificacaoIbipora.shp',
        'areas': {0: '1.52 (3.25%)', 1: '2.62 (5.61%)', 2: '12.44 (26.68%)', 3: '7.83 (16.79%)', 
                 4: '0.0 (0.0%)', 5: '3.97 (8.51%)', 6: '9.95 (21.35%)', 7: '1.78 (3.82%)', 
                 8: '6.5 (13.94%)', 9: '0.02 (0.05%)'},
        'total': '46.63'
    },
    'Jandaia': {
        'path': '/Volumes/ssd_externo/UEL DOUTORADO 2022/Artigo CLIP GEO/Jandaia/classificacaoJandaia.shp',
        'areas': {0: '1.89 (9.33%)', 1: '0.31 (1.54%)', 2: '7.78 (38.45%)', 3: '0.56 (2.78%)', 
                 4: '0.16 (0.77%)', 5: '1.78 (8.8%)', 6: '3.01 (14.88%)', 7: '2.58 (12.76%)', 
                 8: '2.14 (10.57%)', 9: '0.02 (0.12%)'},
        'total': '20.23'
    },
    'Londrina': {
        'path': '/Volumes/ssd_externo/UEL DOUTORADO 2022/Artigo CLIP GEO/Londrina/classificacaoLondrina.shp',
        'areas': {0: '12.02 (5.56%)', 1: '6.68 (3.09%)', 2: '45.33 (20.96%)', 3: '9.51 (4.4%)', 
                 4: '4.22 (1.95%)', 5: '19.97 (9.24%)', 6: '54.52 (25.21%)', 7: '33.71 (15.59%)', 
                 8: '28.26 (13.07%)', 9: '2.0 (0.92%)'},
        'total': '216.23'
    },
    'Maringa': {
        'path': '/Volumes/ssd_externo/UEL DOUTORADO 2022/Artigo CLIP GEO/Maringa/classificacaoMaringa.shp',
        'areas': {0: '4.66 (3.4%)', 1: '2.61 (1.9%)', 2: '21.01 (15.31%)', 3: '4.09 (2.98%)', 
                 4: '2.43 (1.77%)', 5: '23.02 (16.78%)', 6: '47.24 (34.42%)', 7: '20.52 (14.96%)', 
                 8: '11.41 (8.32%)', 9: '0.23 (0.17%)'},
        'total': '137.21'
    },
    'Rolandia': {
        'path': '/Volumes/ssd_externo/UEL DOUTORADO 2022/Artigo CLIP GEO/Rolandia/classificacaoRolandia.shp',
        'areas': {0: '2.09 (4.64%)', 1: '1.09 (2.41%)', 2: '13.35 (29.62%)', 3: '4.39 (9.73%)', 
                 4: '0.02 (0.05%)', 5: '4.83 (10.71%)', 6: '10.03 (22.25%)', 7: '4.79 (10.63%)', 
                 8: '4.22 (9.36%)', 9: '0.26 (0.58%)'},
        'total': '45.07'
    },
    'Mandaguari': {
        'path': '/Volumes/ssd_externo/UEL DOUTORADO 2022/Artigo CLIP GEO/Mandaguari/classificacaoMandaguari.shp',
        'areas': {0: '3.97 (9.43%)', 1: '0.67 (1.59%)', 2: '17.11 (40.68%)', 3: '4.31 (10.26%)', 
                 4: '0.0 (0.0%)', 5: '3.36 (7.98%)', 6: '6.14 (14.6%)', 7: '1.95 (4.63%)', 
                 8: '4.42 (10.51%)', 9: '0.13 (0.31%)'},
        'total': '42.07'
    },
    'Marialva': {
        'path': '/Volumes/ssd_externo/UEL DOUTORADO 2022/Artigo CLIP GEO/Marialva/classificacaoMarialva.shp',
        'areas': {0: '2.01 (7.31%)', 1: '0.62 (2.26%)', 2: '9.44 (34.39%)', 3: '2.98 (10.84%)', 
                 4: '0.0 (0.0%)', 5: '3.38 (12.32%)', 6: '6.74 (24.55%)', 7: '1.21 (4.4%)', 
                 8: '1.04 (3.79%)', 9: '0.04 (0.13%)'},
        'total': '27.45'
    }
}

# Carregar todos os shapefiles
all_bounds = []
city_gdfs = {}


print("Carregando shapefiles...")

for city_name, city_info in cities_data.items():
    try:
        gdf = gpd.read_file(city_info['path'])
        
        if gdf.crs != 'EPSG:4326':
            gdf = gdf.to_crs('EPSG:4326')
        
        gdf['municipio'] = city_name
        
        # Adicionar cor e label baseado na classe
        gdf['color'] = gdf['pred'].map(class_colors)
        gdf['class_label'] = gdf['pred'].map(class_labels)
        
        city_gdfs[city_name] = gdf
        all_bounds.extend(gdf.total_bounds)
        
        print(f"✓ {city_name}: {len(gdf)} polígonos")
        
    except Exception as e:
        print(f"✗ Erro ao carregar {city_name}: {e}")

# Calcular centro
if all_bounds:
    center_lat = (min(all_bounds[1::4]) + max(all_bounds[3::4])) / 2
    center_lon = (min(all_bounds[0::4]) + max(all_bounds[2::4])) / 2
else:
    center_lat, center_lon = -23.4, -51.5

print(f"Centro do mapa: {center_lat:.4f}, {center_lon:.4f}")

# Criar mapa base
m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=10,
    tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
    attr='Google Satellite'
)

# OTIMIZAÇÃO: Adicionar cada cidade como UM ÚNICO GeoJSON
print("\nAdicionando cidades ao mapa...")

for city_name, gdf in city_gdfs.items():
    # Converter GeoDataFrame inteiro para GeoJSON
    geojson_data = json.loads(gdf.to_json())
    
    # Criar grupo de camada
    city_group = folium.FeatureGroup(name=city_name, show=True)
    
    # Adicionar GeoJSON único para toda a cidade
    GeoJson(
        geojson_data,
        style_function=lambda feature: {
            'fillColor': feature['properties']['color'],
            'color': feature['properties']['color'],
            'weight': 0,
            'opacity': 0,
            'fillOpacity': 0.7
        },
        popup=folium.GeoJsonPopup(
            fields=['municipio', 'class_label'],
            aliases=['City:', 'Class:'],
            localize=True
        ),
        tooltip=folium.GeoJsonTooltip(
            fields=['municipio', 'class_label'],
            aliases=['', ''],
            localize=True
        )
    ).add_to(city_group)
    
    city_group.add_to(m)
    print(f"✓ Adicionado: {city_name}")

# Controle de camadas
folium.LayerControl(position='topleft', collapsed=False).add_to(m)

# Legenda
legend_html = '''
<div style="position: fixed; 
            top: 10px; right: 10px; width: 300px; max-height: 90vh; 
            background-color: white; border: 2px solid grey; z-index: 9999; 
            font-size: 10px; padding: 10px; border-radius: 5px; 
            overflow-y: auto; box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
<h4 style="margin: 0 0 10px 0; text-align: center; color: #2c5282;">Land Cover Classification</h4>
<p style="margin: 0 0 10px 0; text-align: center; font-size: 9px; color: #666;">Area in km² (%)</p>
'''

for city_name, city_info in cities_data.items():
    legend_html += f'<div style="margin-bottom: 12px; border-bottom: 1px solid #eee; padding-bottom: 8px;">'
    legend_html += f'<b style="color: #2c5282;">{city_name}</b> <span style="color: #666;">({city_info["total"]} km²)</span><br>'
    
    for class_id in sorted(class_colors.keys()):
        if class_id in city_info['areas']:
            color = class_colors[class_id]
            label = class_labels[class_id]
            area_info = city_info['areas'][class_id]
            legend_html += f'<div style="margin: 2px 0;"><span style="color:{color}; font-size: 14px;">■</span> <span style="font-size: 9px;">{label}: {area_info}</span></div>'
    
    legend_html += '</div>'

legend_html += '''
<div style="margin-top: 10px; padding-top: 10px; border-top: 2px solid #2c5282; font-size: 9px; color: #666;">
<b>Specifications:</b><br>
- Model: CLIP-ViT-B/32<br>
- Year: 2024<br>
- Classes: 10 land cover types
</div>
</div>
'''

m.get_root().html.add_child(folium.Element(legend_html))

# Salvar
output_path = "/Users/fjcosta/Documents/landCoverlandValue/landcover/landcoverCLIP_NPr.html"
m.save(output_path)

# Verificar tamanho
import os
file_size_mb = os.path.getsize(output_path) / (1024 * 1024)

print(f"\n✅ Mapa salvo: {output_path}")
print(f"📊 Cidades: {len(city_gdfs)}")
print(f"📍 Polígonos: {sum(len(gdf) for gdf in city_gdfs.values())}")
print(f"📦 Tamanho: {file_size_mb:.2f} MB")

if file_size_mb > 60:
    print(f"⚠️ Arquivo maior que 60 MB!")

Carregando shapefiles...
